# Team EGE - BLG453E Term Project
Egehan Orta - 150160124

Yavuz Ege Okumuş - 150160118
## Bouncing Balls

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import moviepy.editor as mpy
from scipy import signal

In [2]:
movie = mpy.VideoFileClip('./movie_001.avi')

In [3]:
pure_blue = np.uint8([[[73,86,236]]])
hsv_blue = cv.cvtColor(pure_blue, cv.COLOR_RGB2HSV)
ub = hsv_blue + [[[10, 0 , 0]]]
lb = hsv_blue + [[[-10, -155 , -155]]]

pure_green = np.uint8([[[35,208,90]]])
hsv_green = cv.cvtColor(pure_green, cv.COLOR_RGB2HSV)
ug = hsv_green + [[[10, 0 , 0]]]
lg = hsv_green + [[[-10, -155 , -155]]]

pure_red = np.uint8([[[253,33,41]]])
hsv_red = cv.cvtColor(pure_red, cv.COLOR_RGB2HSV)
ur = hsv_red + [[[10, 0 , 0]]]
lr = hsv_red + [[[-10, -155 , -155]]]

pure_pink = np.uint8([[[253,203,236]]])
hsv_pink = cv.cvtColor(pure_pink, cv.COLOR_RGB2HSV)
up = hsv_pink + [[[10, 0 , 0]]]
lp = hsv_pink + [[[-10, -155 , -155]]]

In [21]:
def optical_flow(im1, im2, window_size, points):
    im1_org = im1.copy()
    im2_org = im2.copy()
    
    im1 = cv.cvtColor(im1, cv.COLOR_RGB2GRAY)
    im2 = cv.cvtColor(im2, cv.COLOR_RGB2GRAY)
    
    w = window_size//2
    im1 = im1 / 255.0
    im2 = im2 / 255.0
    
    ix = conv(im1, kernel_x, 'same') + conv(im2, kernel_x, 'same')
    iy = conv(im1, kernel_y, 'same') + conv(im2, kernel_y, 'same')
    it = conv(im1, kernel_t, 'same') + conv(im2, -kernel_t, 'same')
    
    uv_list = []
    img = im1_org.copy()
    for point in points:
        x,y = point
    
        ixx = ix[y-w:y+w+1, x-w:x+w+1].ravel().reshape((-1,1))
        iyy = iy[y-w:y+w+1, x-w:x+w+1].ravel().reshape((-1,1))
        
        a = np.dstack((ixx.ravel(),iyy.ravel())).reshape(-1,2)
        b = np.array([ it[y-w:y+w+1, x-w:x+w+1] ]).ravel().reshape((-1,1))
        ata = np.matmul(a.T,a)
        atb = np.matmul(a.T,b)
        atainv = np.linalg.pinv(ata)
        uv = -np.matmul(atainv,atb)
        uv_list.append(uv)
        
        if (uv == 0).all():
            uvn = uv.copy()
        else:
            uvn = 20 * uv / np.linalg.norm(uv)
        
        img = cv.arrowedLine(img, (x,y), (x+int(uvn[0]), y+int(uvn[1])), (255,0,150), 2, tipLength=0.8)
    return img, uv_list

In [5]:
kernel_x = np.array([[-1., 1.], [-1., 1.]]) * 0.25
kernel_y = np.array([[-1., -1.], [1., 1.]]) * 0.25
kernel_t = np.array([[1., 1.], [1., 1.]]) * 0.25
kernel = np.uint8([[0,1,0],[1,1,1],[0,1,0]])
conv = signal.convolve2d

In [81]:
def mask_creator(img):
    img = img.copy()
    img = cv.blur(img,(3,3))
    img = cv.blur(img,(3,3))
    img = cv.morphologyEx(img, cv.MORPH_CLOSE, np.ones((3,3)),iterations=3)
    
    hsv = cv.cvtColor(img, cv.COLOR_RGB2HSV)
    blue_mask = cv.inRange(hsv, lb, ub)
    green_mask = cv.inRange(hsv, lg, ug)
    red_mask = cv.inRange(hsv, lr, ur)
    pink_mask = cv.inRange(hsv, lp, up)
    
    blue_mask = cv.morphologyEx(blue_mask, cv.MORPH_CLOSE, kernel,iterations=5)
    blue_mask = cv.morphologyEx(blue_mask, cv.MORPH_OPEN, kernel,iterations=2)
    blue_mask = cv.morphologyEx(blue_mask, cv.MORPH_CLOSE, kernel,iterations=2)
    
    green_mask = cv.morphologyEx(green_mask, cv.MORPH_CLOSE, kernel,iterations=5)
    green_mask = cv.morphologyEx(green_mask, cv.MORPH_OPEN, kernel,iterations=2)
    green_mask = cv.morphologyEx(green_mask, cv.MORPH_CLOSE, kernel,iterations=2)
    
    red_mask = cv.morphologyEx(red_mask, cv.MORPH_CLOSE, kernel,iterations=5)
    red_mask = cv.morphologyEx(red_mask, cv.MORPH_OPEN, kernel,iterations=2)
    red_mask = cv.morphologyEx(red_mask, cv.MORPH_CLOSE, kernel,iterations=2)
    
    pink_mask = cv.morphologyEx(pink_mask, cv.MORPH_OPEN, kernel,iterations=4)
    pink_mask = cv.morphologyEx(pink_mask, cv.MORPH_CLOSE, kernel,iterations=5)
#     pink_mask = cv.morphologyEx(pink_mask, cv.MORPH_CLOSE, kernel,iterations=2)
#     mask = blue_mask + green_mask + red_mask + pink_mask

    mid_points = []
    for i in [blue_mask, green_mask, red_mask, pink_mask]:
        y,x = np.nonzero(i)
        mid_points.append((int(np.round(np.mean(x))), int(np.round(np.mean(y)))))

    
    return mid_points

In [82]:
n_frames = movie.reader.nframes
fps = movie.fps

In [84]:
ims = []
uvs = []
for i in range(n_frames-1):
    im1 = movie.get_frame(i*1.0/fps)
    im2 = movie.get_frame((i+1)*1.0/fps)
    
    points = mask_creator(im1)
    im,uv = optical_flow(im1,im2,25,points)
    ims.append(im)
    uvs.append(uv)

In [85]:
clip = mpy.ImageSequenceClip(ims, fps = fps)
clip.write_videofile("part3_team_ege_output.mp4", codec="libx264")

Moviepy - Building video part3_team_ege_output.mp4.
Moviepy - Writing video part3_team_ege_output.mp4



Moviepy - Done !
Moviepy - video ready part3_team_ege_output.mp4


In [86]:
sums = {
    'blue':0,
    'green':0,
    'red':0,
    'pink':0
}

for i in range(len(uvs)):
    for j,key in enumerate(sums.keys()):
        disp = uvs[i][j]
        sums[key] = sums[key] + np.sqrt(np.sum(disp**2))

In [88]:
for i in sums.keys():
    print('Velocity of',i,'=',sums[i]/n_frames,'px/frame')
    print('Velocity of',i,'=',sums[i]/movie.duration,'px/sec')
    print()

Velocity of blue = 2.860059371782575 px/frame
Velocity of blue = 85.75427186158718 px/sec

Velocity of green = 2.8947329495677674 px/frame
Velocity of green = 86.79390322209004 px/sec

Velocity of red = 4.179407995564518 px/frame
Velocity of red = 125.31281448494943 px/sec

Velocity of pink = 2.257580799599167 px/frame
Velocity of pink = 67.68992264578898 px/sec

